<a href="https://colab.research.google.com/github/missLaiba22/Rag-docs/blob/main/world_history_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install langchain langchain-community openai tiktoken pinecone-client langchain_pinecone unstructured pdfminer==20191125 pdfminer.six==20221105 pytube sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 21.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 28.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nv

In [2]:
!pip install openai==0.27.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 1.7 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.40.3
    Uninstalling openai-1.40.3:
      Successfully uninstalled openai-1.40.3


In [3]:
from langchain.document_loaders import PyPDFLoader  # For loading PDF documents
from langchain.text_splitter import RecursiveCharacterTextSplitter  # For splitting text into manageable chunks
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
import pinecone  # For interacting with Pinecone vector database
import openai  # For using OpenAI's GPT-4 API
import os  # For setting environment variables


In [4]:
from google.colab import userdata
pinecone_api_key = userdata.get("PINECONE_API_KEY")
os.environ['PINECONE_API_KEY'] = pinecone_api_key

In [5]:
# Set the OpenRouter API key
openrouter_api_key = os.getenv("OPENROUTER_API_KEY")

# Initialize the OpenAI client using OpenRouter
openai.api_key = openrouter_api_key
openai.api_base = "https://openrouter.ai/api/v1"


In [6]:
from langchain.document_loaders import PyPDFLoader

def extract_text_from_pdf(pdf_path):
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()
    # Combine all text from the pages into one large text
    full_text = " ".join([doc.page_content for doc in documents])
    return full_text

pdf_text = extract_text_from_pdf("/content/01. World History. Cultures, States, and Societies to 1500 author Eugene Berger, George L. Israel, Charlotte Miller, Brian Parkinson, Andrew Reeves and Nadejda Williams.pdf")


In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import tiktoken

tokenizer = tiktoken.get_encoding('p50k_base')

def tiktoken_len(text):
    tokens = tokenizer.encode(text, disallowed_special=())
    return len(tokens)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,  # Adjust based on your needs and the model's token limit
    chunk_overlap=100,
    length_function=tiktoken_len,
    separators=["\n\n", "\n", " ", ""]
)

chunks = text_splitter.split_text(pdf_text)

print(f"Number of chunks: {len(chunks)}")
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1} size: {tiktoken_len(chunk)} tokens")


Number of chunks: 150
Chunk 1 size: 1994 tokens
Chunk 2 size: 1969 tokens
Chunk 3 size: 1975 tokens
Chunk 4 size: 1990 tokens
Chunk 5 size: 1939 tokens
Chunk 6 size: 1967 tokens
Chunk 7 size: 1920 tokens
Chunk 8 size: 1828 tokens
Chunk 9 size: 1878 tokens
Chunk 10 size: 1928 tokens
Chunk 11 size: 1957 tokens
Chunk 12 size: 1979 tokens
Chunk 13 size: 1969 tokens
Chunk 14 size: 1960 tokens
Chunk 15 size: 1951 tokens
Chunk 16 size: 1790 tokens
Chunk 17 size: 1980 tokens
Chunk 18 size: 1884 tokens
Chunk 19 size: 1902 tokens
Chunk 20 size: 1961 tokens
Chunk 21 size: 1944 tokens
Chunk 22 size: 1993 tokens
Chunk 23 size: 1986 tokens
Chunk 24 size: 1993 tokens
Chunk 25 size: 1978 tokens
Chunk 26 size: 1992 tokens
Chunk 27 size: 1974 tokens
Chunk 28 size: 1983 tokens
Chunk 29 size: 1990 tokens
Chunk 30 size: 1947 tokens
Chunk 31 size: 1988 tokens
Chunk 32 size: 1977 tokens
Chunk 33 size: 1994 tokens
Chunk 34 size: 1980 tokens
Chunk 35 size: 1987 tokens
Chunk 36 size: 1998 tokens
Chunk 37 size: 

In [8]:
!pip install transformers


In [9]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

# Load the model and tokenizer
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def generate_embeddings(texts):
    embeddings = []
    for text in texts:
        # Tokenize the text
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

        # Generate embeddings
        with torch.no_grad():
            outputs = model(**inputs)
            # Get the mean of the token embeddings as the sentence embedding
            sentence_embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
            embeddings.append(sentence_embedding)
    return embeddings

# Generate embeddings for each chunk
embeddings = generate_embeddings(chunks)

# Print the shape of embeddings
print(f"Number of embeddings: {len(embeddings)}")
print(f"Embedding shape: {embeddings[0].shape if embeddings else 'No embeddings generated'}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

Number of embeddings: 150
Embedding shape: (384,)


In [10]:
!pip install pinecone-client
!pip install protoc-gen-openapiv2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.3/309.3 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.27.3 which is incompatible.
google-ai-generativelanguage 0.6.6 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.27.3 which is incompatible.
google-cloud-aiplatform 1.59.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.27.3 which is incompatible.
google-cloud-bigquery-storage 2.25.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.

In [11]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os
index_name = "world-history"
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name="world-history",
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

In [12]:
# Example: Assuming `embeddings` is a list of tuples (id, vector)
# id: Unique identifier for each vector
# vector: The embedding vector
vectors_to_upsert = [(f"id_{i}", embedding) for i, embedding in enumerate(embeddings)]


In [13]:
# Connect to the index
index = pc.Index("world-history")

# Upsert the vectors
index.upsert(vectors=vectors_to_upsert)

print(f"Upserted {len(vectors_to_upsert)} vectors into the index.")


Upserted 150 vectors into the index.


In [17]:


def get_embeddings(texts):
    response = openai.Embedding.create(input=texts, model="text-embedding-3-small")
    return [embedding['embedding'] for embedding in response['data']]

In [18]:
def perform_rag(query):
    # Generate the query embedding
    query_embedding = get_embeddings([query])[0]

    # Perform the Pinecone query
    top_matches = index.query(vector=query_embedding, top_k=10, include_metadata=True)

    # Get the list of retrieved texts
    contexts = [item['metadata']['text'] for item in top_matches['matches']]

    # Construct the augmented query
    augmented_query = "<CONTEXT>\n" + "\n\n-------\n\n".join(contexts[:10]) + "\n-------\n</CONTEXT>\n\n\n\nMY QUESTION:\n" + query

    # Define a relevant system prompt
    system_prompt = (
        "You are an expert on the book titled 'World History: Cultures, States, and Societies to 1500' "
        "by Eugene Berger, George L. Israel, Charlotte Miller, Brian Parkinson, Andrew Reeves, and Nadejda Williams. "
        "Answer questions based only on the provided context from this book."
    )
    # Perform the query with OpenRouter
    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": augmented_query}
        ]
    )

    return response.choices[0].message["content"]

In [ ]:
!pip install openai==0.27.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.40.3
    Uninstalling openai-1.40.3:
      Successfully uninstalled openai-1.40.3


In [19]:
print(perform_rag("What were the key cultural developments in the Mediterranean region before 1500?"))
print(perform_rag("How did the ancient civilizations of Mesopotamia influence subsequent societies?"))

AuthenticationError: No API key provided. You can set your API key in code using 'openai.api_key = <API-KEY>', or you can set the environment variable OPENAI_API_KEY=<API-KEY>). If your API key is stored in a file, you can point the openai module at it with 'openai.api_key_path = <PATH>'. You can generate API keys in the OpenAI web interface. See https://onboard.openai.com for details, or email support@openai.com if you have any questions.